In [1]:
import cv2
from cv2 import aruco
import numpy as np
import pandas as pd
import os
import glob

In [2]:
# 체커보드의 차원 정의
CHECKERBOARD = (6,8) # 체커보드 행과 열당 내부 코너 수
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# 각 체커보드 이미지에 대한 3D 점 벡터를 저장할 벡터 생성
objpoints = []
# 각 체커보드 이미지에 대한 2D 점 벡터를 저장할 벡터 생성
imgpoints = [] 
# 3D 점의 세계 좌표 정의
objp = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
prev_img_shape = None

In [3]:
# 주어진 디렉터리에 저장된 개별 이미지의 경로 추출
images = glob.glob('./images/*.jpg')
for fname in images:
    img = cv2.imread(fname)
    # 그레이 스케일로 변환
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # 체커보드 코너 찾기
    # 이미지에서 원하는 개수의 코너가 발견되면 ret = true
    ret, corners = cv2.findChessboardCorners(gray,
                                             CHECKERBOARD,
                                             cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
    # 원하는 개수의 코너가 감지되면,
    # 픽셀 좌표 미세조정 -> 체커보드 이미지 표시
    if ret == True:
        objpoints.append(objp)
        # 주어진 2D 점에 대한 픽셀 좌표 미세조정
        corners2 = cv2.cornerSubPix(gray, corners, (11,11),(-1,-1), criteria)
        imgpoints.append(corners2)
        # 코너 그리기 및 표시
        img = cv2.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)
    cv2.imshow('img',img)
    cv2.waitKey(0)
cv2.destroyAllWindows()
h,w = img.shape[:2] # 480, 640
# 알려진 3D 점(objpoints) 값과 감지된 코너의 해당 픽셀 좌표(imgpoints) 전달, 카메라 캘리브레이션 수행
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

In [4]:
print("Camera matrix : \n") # 내부 카메라 행렬
print(mtx)

Camera matrix : 

[[5.69388816e+03 0.00000000e+00 1.84375874e+03]
 [0.00000000e+00 5.69616921e+03 1.13412874e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [5]:
print("dist : \n") # 렌즈 왜곡 계수(Lens distortion coefficients)
print(dist)

dist : 

[[-5.75322844e-01 -3.55776935e-02 -1.26358540e-02 -4.40453397e-03
   3.01122901e+01]]


In [6]:
print("rvecs : \n") # 회전 벡터
print(rvecs)

rvecs : 

(array([[-0.15152656],
       [ 0.05718157],
       [ 1.55664856]]), array([[-0.12212314],
       [ 0.09727981],
       [ 1.46734996]]), array([[-0.12442954],
       [ 0.06170135],
       [ 1.51621589]]), array([[-0.16231616],
       [ 0.05150364],
       [ 1.55569665]]))


In [7]:
print("tvecs : \n") # 이동 벡터
print(tvecs)

tvecs : 

(array([[ 2.34848404],
       [-3.45916829],
       [27.98409241]]), array([[ 3.56017269],
       [-3.38822809],
       [27.49144807]]), array([[ 2.88511263],
       [-3.86851882],
       [27.76812337]]), array([[ 2.3281809 ],
       [-3.20067509],
       [28.05792016]]))


In [8]:
def createDirectory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")

In [9]:
# undistort
dir="images"
createDirectory("undistort_chesboard")

for i in os.listdir(dir):
    img=cv2.imread(dir+'/'+i)
    img_undist = cv2.undistort(img, mtx, dist, None)
    concatenated_image = cv2.hconcat([img, img_undist])
    concatenated_image = cv2.resize(concatenated_image, (0,0), fx=0.5, fy=0.5)
    cv2.imshow("undistorted", concatenated_image)
    cv2.imwrite("undistort_chesboard/undist_{0}.png".format(i), concatenated_image)
    key = cv2.waitKey(0)
    if key == 27:
        cv2.destroyAllWindows()